In [269]:
from pathlib import Path
import pandas as pd

from pam.read.matsim import load_attributes_map_from_v12, stream_matsim_persons
from pam.utils import datetime_to_matsim_time
from pam.core import Population, Person
from pam.activity import Activity, Plan, Leg

In [270]:
dir = Path("C:/Users/ucfnfjs/Data/2019_baseline_re_run_20221209_low_cost_bus_walk")
input_plans = dir / "input_plans.xml"
iter_50 = dir / "output_experienced_plans_ITER50.xml" / "output_experienced_plans_ITER50.xml"
iter_100 = dir / "output_experienced_plans_ITER100.xml" / "output_experienced_plans_ITER100.xml"
iter_150 = dir / "output_experienced_plans_ITER150.xml" / "output_experienced_plans_ITER150.xml"
iter_200 = dir / "output_experienced_plans.xml_ITER200" / "output_experienced_plans.xml_ITER200"
iter_200.exists()

True

In [271]:
streamer0 = stream_matsim_persons(input_plans, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
streamer1 = stream_matsim_persons(iter_50, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
streamer2 = stream_matsim_persons(iter_100, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
streamer3 = stream_matsim_persons(iter_150, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
streamer4 = stream_matsim_persons(iter_200, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
streamers = [streamer1, streamer2, streamer3, streamer4]


In [272]:
def dt_to_min(dt) -> int:
    h, m, s = datetime_to_matsim_time(dt).split(":")
    return (int(h) * 60) + int(m)


def person_to_schedule(person: Person) -> tuple:
    score = person.plan.score
    if score is None:
        score = 0
    record = []
    for component in person.plan:
        if isinstance(component, Leg):
            distance = component.euclidean_distance
            mode = component.mode
        else:
            distance = 0
            mode = "NA"
        record.append(
            [
                component.act,
                dt_to_min(component.start_time),
                dt_to_min(component.end_time),
                mode,
                distance
            ]
        )
    return (person.pid, (score, [record]))


def pam_to_schedules(population: Population) -> dict:
    return dict([person_to_schedule(person) for person in population])


def add_pid(record, pid):
    record = [[pid] + line for line in record]
    return record

In [273]:
best = pam_to_schedules(streamer0)

lhss = []
rhss = []
mapper = {}
i = 0

for s, streamer in enumerate(streamers):
    print("stream: " ,s)
    for person in streamer:
        pid, (score, (record,)) = person_to_schedule(person)
        existing_score, existing_records = best[pid]
        if score > existing_score:
            new_records = existing_records + [record]
            for existing in existing_records:
                try:
                    lhss.extend(add_pid(existing, i))
                    rhss.extend(add_pid(record, i))
                    mapper[i] = pid
                    i += 1
                except:
                    print(existing)
                    print(record)
                    raise
            best[pid] = (score, new_records)


stream:  0
stream:  1
stream:  2
stream:  3


In [274]:
lhs = pd.DataFrame(lhss, columns=["pid", "act", "start", "end", "mode", "distance"])
rhs = pd.DataFrame(rhss, columns=["pid", "act", "start", "end", "mode", "distance"])
print(len(lhs.pid.unique()), len(rhs.pid.unique()))
print(lhs.head(20))
print()
print(rhs.head(20))

621856 621856
    pid     act  start   end mode  distance
0     0    home      0   625   NA  0.000000
1     0  travel    625   650  bus  0.000000
2     0   other    650   780   NA  0.000000
3     0  travel    780   805  bus  0.000000
4     0    home    805  1440   NA  0.000000
5     1    home      0   625   NA  0.000000
6     1  travel    625   650  bus  0.000000
7     1   other    650   780   NA  0.000000
8     1  travel    780   805  bus  0.000000
9     1    home    805  1440   NA  0.000000
10    2    home      0   570   NA  0.000000
11    2  travel    570   595  bus  0.401693
12    2    shop    595   665   NA  0.000000
13    2  travel    665   685  bus  0.401693
14    2    home    685  1440   NA  0.000000
15    3    home      0   570   NA  0.000000
16    3  travel    570   595  bus  1.932700
17    3    shop    595   665   NA  0.000000
18    3  travel    665   685  bus  1.932700
19    3    home    685  1440   NA  0.000000

    pid     act  start   end  mode  distance
0     0    home 

In [275]:
len(attributes)

116039

In [276]:
attributes = load_attributes_map_from_v12(input_plans)
attributes = pd.DataFrame(attributes).T
attributes = attributes[-attributes.subpopulation.isin(["lgv", "hgv"])]
attributes.head()

,subpopulation,CarType,CarCO2,hid,hid_old,hzone,householdid,hcounty,individualid,surveyyear,gender,age,workstatus,hasLicence,hasCar,hasBike,car_avail,hhincome,sex,age_group
2002000535_3_E02001519_1_2002001259,car_avail_no_low,NaN,NaN,2002000535_3_E02001519_1_2002001259,2002000535_3,E02001519,2002000535,Barnsley,2002001259,2002,male,82,inactive,no,True,False,never,low,m,80 to 84
2002000535_3_E02001519_1_2002001260,car_avail_no_low,NaN,NaN,2002000535_3_E02001519_1_2002001260,2002000535_3,E02001519,2002000535,Barnsley,2002001260,2002,female,82,inactive,no,True,False,never,low,f,80 to 84
2002000535_4_E02001553_1_2002001259,car_avail_no_low,NaN,NaN,2002000535_4_E02001553_1_2002001259,2002000535_4,E02001553,2002000535,Doncaster,2002001259,2002,male,82,inactive,no,True,False,never,low,m,80 to 84
2002000535_4_E02001553_1_2002001260,car_avail_no_low,NaN,NaN,2002000535_4_E02001553_1_2002001260,2002000535_4,E02001553,2002000535,Doncaster,2002001260,2002,female,82,inactive,no,True,False,never,low,f,80 to 84
2002000535_4_E02001571_1_2002001259,car_avail_no_low,NaN,NaN,2002000535_4_E02001571_1_2002001259,2002000535_4,E02001571,2002000535,Doncaster,2002001259,2002,male,82,inactive,no,True,False,never,low,m,80 to 84


In [277]:
streamer = stream_matsim_persons(input_plans, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
for person in streamer:
    modes = [l.mode for l in person.plan.legs]
    if "bus" in modes:
        for l in person.plan.legs:
            print(l.mode)
            print(l.distance)
        print(person.plan.score)


bus
0.0
bus
0.0
None
bus
0.0
bus
0.0
None
bus
401.69266859130005
bus
401.69266859130005
None
bus
1932.6999249754215
bus
1932.6999249754215
None
bus
5956.958703902521
bus
5956.958703902521
None
bus
8749.46604085072
bus
8749.46604085072
None
bus
2998.6205161707276
bus
0.0
car
2488.3852193742027
None
car
5391.754074510446
bus
11677.76776614435
bus
9084.011944069647
car
6891.0653022591505
None
car
7956.622650346062
bus
4943.1222926405535
bus
4908.377532342026
car
4765.018887685546
None
car
6490.239209767234
bus
6592.687539994597
bus
6403.070591520915
car
5786.731374446199
None
car
4771.017711138788
bus
5688.000175808717
bus
5177.593456423553
car
2524.545107539178
None
car
5940.88924320257
bus
7839.782458716568
bus
8349.711492021745
car
2479.7891039360584
None
bus
4455.362611505376
bus
4455.362611505376
None
bus
6549.997938930974
bus
6549.997938930974
None
bus
6069.11237332116
bus
6069.11237332116
None
bus
1762.6689989898841
bus
1762.6689989898841
None
bus
258.69866640553056
bus
258.6986664

In [278]:
streamer = stream_matsim_persons(iter_50, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
for person in streamer:
    modes = [l.mode for l in person.plan.legs]
    if "rail" in modes:
        person.print()
        print(person.plan.score)
        break

Person: 2002000535_4_E02001571_1_2002001260
{}
0:	Activity(act:home, location:POINT (453134 394839), time:00:00:00 --> 09:30:00, duration:9:30:00)
1:	Leg(mode:rail, area:POINT (453134 394839) --> POINT (459100 401239), time:09:30:00 --> 10:35:58, duration:1:05:58)
2:	Activity(act:shop, location:POINT (459100 401239), time:10:35:58 --> 11:05:00, duration:0:29:02)
3:	Leg(mode:bus, area:POINT (459100 401239) --> POINT (453134 394839), time:11:05:00 --> 11:50:47, duration:0:45:47)
4:	Activity(act:home, location:POINT (453134 394839), time:11:50:47 --> 00:00:00, duration:12:09:13)
143.10307922106023


In [279]:
streamer = stream_matsim_persons(iter_100, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
for person in streamer:
    modes = [l.mode for l in person.plan.legs]
    if "bus" in modes:
        person.print()
        print(person.plan.score)
        break

Person: 2002000535_4_E02001553_1_2002001259
{}
0:	Activity(act:home, location:POINT (459109 404898), time:00:00:00 --> 09:37:07, duration:9:37:07)
1:	Leg(mode:bus, area:POINT (459109 404898) --> POINT (459375 405199), time:09:37:07 --> 09:42:37, duration:0:05:30)
2:	Activity(act:shop, location:POINT (459375 405199), time:09:42:37 --> 10:52:09, duration:1:09:32)
3:	Leg(mode:walk, area:POINT (459375 405199) --> POINT (459109 404898), time:10:52:09 --> 11:00:36, duration:0:08:27)
4:	Activity(act:home, location:POINT (459109 404898), time:11:00:36 --> 00:00:00, duration:12:59:24)
189.803144971215


In [280]:
streamer = stream_matsim_persons(iter_150, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
for person in streamer:
    modes = [l.mode for l in person.plan.legs]
    if "bus" in modes:
        person.print()
        print(person.plan.score)
        break

Person: 2002000535_4_E02001553_1_2002001259
{}
0:	Activity(act:home, location:POINT (459109 404898), time:00:00:00 --> 09:37:07, duration:9:37:07)
1:	Leg(mode:bus, area:POINT (459109 404898) --> POINT (459375 405199), time:09:37:07 --> 09:42:37, duration:0:05:30)
2:	Activity(act:shop, location:POINT (459375 405199), time:09:42:37 --> 10:52:09, duration:1:09:32)
3:	Leg(mode:walk, area:POINT (459375 405199) --> POINT (459109 404898), time:10:52:09 --> 11:00:36, duration:0:08:27)
4:	Activity(act:home, location:POINT (459109 404898), time:11:00:36 --> 00:00:00, duration:12:59:24)
189.803144971215


In [281]:
streamer = stream_matsim_persons(iter_200, simplify_pt_trips=True, keep_non_selected=True, leg_attributes=True, leg_route=True)
for person in streamer:
    modes = [l.mode for l in person.plan.legs]
    if "bus" in modes:
        person.print()
        print(person.plan.score)
        break

Person: 2002000535_4_E02001553_1_2002001259
{}
0:	Activity(act:home, location:POINT (459109 404898), time:00:00:00 --> 09:37:07, duration:9:37:07)
1:	Leg(mode:bus, area:POINT (459109 404898) --> POINT (459375 405199), time:09:37:07 --> 09:42:37, duration:0:05:30)
2:	Activity(act:shop, location:POINT (459375 405199), time:09:42:37 --> 10:52:09, duration:1:09:32)
3:	Leg(mode:walk, area:POINT (459375 405199) --> POINT (459109 404898), time:10:52:09 --> 11:00:36, duration:0:08:27)
4:	Activity(act:home, location:POINT (459109 404898), time:11:00:36 --> 00:00:00, duration:12:59:24)
189.803144971215
